# SQL

**Описание проекта**

Задача проанализировать базу данных с информацей о книгах, издательствах, авторах, а также пользовательские обзоры книг.

**Цель исследования**

Сформулировать ценностное предложение для нового продукта.

**Ход исследования**

Необходимо ответить на следующие вопросы:

- Посчитайте, сколько книг вышло после 1 января 2000 года;
- Для каждой книги посчитайте количество обзоров и среднюю оценку;
- Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры;
- Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;
- Посчитайте среднее количество обзоров от пользователей, которые поставили больше 50 оценок.


**Таблица `books`**

Содержит данные о книгах:

- `book_id` — идентификатор книги;
- `author_id` — идентификатор автора;
- `title` — название книги;
- `num_pages` — количество страниц;
- `publication_date` — дата публикации книги;
- `publisher_id` — идентификатор издателя.

**Таблица `authors`**

Содержит данные об авторах:

- `author_id` — идентификатор автора;
- `author` — имя автора.

**Таблица `publishers`**

Содержит данные об издательствах:

- `publisher_id` — идентификатор издательства;
- `publisher` — название издательства;

**Таблица `ratings`**

Содержит данные о пользовательских оценках книг:

- `rating_id` — идентификатор оценки;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, оставившего оценку;
- `rating` — оценка книги.

**Таблица `reviews`**

Содержит данные о пользовательских обзорах на книги:

- `review_id` — идентификатор обзора;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, написавшего обзор;
- `text` — текст обзора.

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import create_engine

In [2]:
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
 db_config['pwd'],
 db_config['host'],
 db_config['port'],
 db_config['db'])
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'}) 

In [3]:
#создадим функцию, возвращающую sql запрос
def sql_result(query):
    return pd.io.sql.read_sql(query, con = engine)

### Исследование таблиц

**Таблица books**

In [4]:
query = \
'''

SELECT * 
FROM books
LIMIT 5

'''
sql_result(query)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


**Таблица authors**

In [5]:
query = \
'''

SELECT * 
FROM authors
LIMIT 5

'''
sql_result(query)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


**Таблица publishers**

In [6]:
query = \
'''

SELECT * 
FROM publishers
LIMIT 5

'''
sql_result(query)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


**Таблица ratings**

In [7]:
query = \
'''

SELECT * 
FROM ratings
LIMIT 5

'''
sql_result(query)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


**Таблица reviews**

In [8]:
query = \
'''

SELECT * 
FROM reviews
LIMIT 5

'''
sql_result(query)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


## Cколько книг вышло после 1 января 2000 года

In [9]:
query = \
'''

SELECT COUNT(book_id)
FROM books
WHERE publication_date > '2000-01-01'

'''
sql_result(query)

,count
0,819


После 1 января 2000 года вышло 819 книг

## Для каждой книги посчитать количество обзоров и среднюю оценку

In [10]:
query = \
'''

SELECT books.title,
       books.book_id,
       COUNT (distinct review_id) as review_cnt,
       AVG(rating) AS avg_rating
FROM books
LEFT JOIN reviews ON reviews.book_id = books.book_id
LEFT JOIN ratings ON ratings.book_id = books.book_id
GROUP BY books.book_id
ORDER BY COUNT(reviews.review_id) DESC

'''
sql_result(query)

,title,book_id,review_cnt,avg_rating
0,Twilight (Twilight #1),948,7,3.662500
1,The Hobbit or There and Back Again,750,6,4.125000
2,The Catcher in the Rye,673,6,3.825581
3,Harry Potter and the Prisoner of Azkaban (Harr...,302,6,4.414634
4,Harry Potter and the Chamber of Secrets (Harry...,299,6,4.287500
...,...,...,...,...
995,Essential Tales and Poems,221,0,4.000000
996,Leonardo's Notebooks,387,0,4.000000
997,The Cat in the Hat and Other Dr. Seuss Favorites,672,0,5.000000
998,Anne Rice's The Vampire Lestat: A Graphic Novel,83,0,3.666667


Мы рассматриваем 999 книг. Из них больше всего обзоров имеет книга Twilight (Twilight #1) - 7

## Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры

In [11]:
query = \
'''

SELECT publisher,
       COUNT (book_id) as book_cnt
FROM books
LEFT JOIN publishers ON publishers.publisher_id=books.publisher_id
WHERE num_pages > 50
GROUP BY publisher
ORDER BY book_cnt DESC

'''
sql_result(query)

,publisher,book_cnt
0,Penguin Books,42
1,Vintage,31
2,Grand Central Publishing,25
3,Penguin Classics,24
4,Ballantine Books,19
...,...,...
329,Turtleback,1
330,Atheneum Books for Young Readers: Richard Jack...,1
331,Penguin Signet,1
332,Victor Gollancz,1


**Penguin Books** и **Vintage** - крупные издательства, которые выпустили более 30 книг	

## Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками

In [12]:
query = \
'''

SELECT author,
       AVG(sub.avg_rating) AS books_avg_rating
FROM (SELECT
     author,
     books.book_id,
     AVG(rating) AS avg_rating
FROM authors
LEFT JOIN books ON books.author_id = authors.author_id
LEFT JOIN ratings ON ratings.book_id = books.book_id
GROUP BY author, 
         books.book_id
HAVING COUNT(rating) > 50) AS sub
GROUP BY author
ORDER BY books_avg_rating DESC
LIMIT 1

'''
sql_result(query)

,author,books_avg_rating
0,J.K. Rowling/Mary GrandPré,4.283844


Автор с самой высокой средней оценкой - **J.K. Rowling**. 

Mary GrandPré - является иллюстратором к книгам о Гарри Поттере

## Посчитайте среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

In [13]:
query = \
'''

SELECT AVG(review_cnt) as avg_review_cnt
FROM (SELECT
     COUNT(review_id) as review_cnt
FROM reviews
WHERE username IN
(SELECT username
FROM ratings
GROUP BY username
HAVING COUNT(ratings) > 50)
GROUP BY username) as sub_review_cnt 

'''
sql_result(query)

,avg_review_cnt
0,24.333333


Среднее количество обзоров от пользователей, которые поставили больше 50 оценок: 24.3.

## Общие выводы:

- После 1 января 2000 года вышло 819 книг
- Книга Twilight (Twilight #1) имеет наибольшее количество обзоров - 7, рейтинг Twilight - 3.7
- издательство, которое выпустило наибольшее число книг толще 50 страниц — Penguin Books. Издательство выпустило 42 книги
- Автор с самой высокой средней оценкой - 4,2 -  J.K. Rowling
- Среднее количество обзоров от пользователей, которые поставили больше 50 оценок: 24.3.